In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
import BingChatAccess
import ChatGPTAccess
import json
import os
from pptx import Presentation

In [73]:
chatGPT = ChatGPTAccess.ChatGPTAccess()

In [74]:
bingChat = BingChatAccess.BingChatAccess()

In [75]:
options = {"len" : 20, "adj" : "historical", "top" : "Roman republic", "foc" : "", "pointsMin" : 4, "pointsMax" : 6}

In [76]:
with open('prompts.json') as f:
    prompts = json.load(f)

In [77]:
first = prompts["outline"].format(**options)
print(first)

Create outline for exhaustive historical presentation about Roman republic. Focus on topics, which are important for the region. For each slide write only brief desriptive name. There hast to be at least 20 slides. Format the list as json. The structure of the json has to be

{
"slides":[
<name of slide 1>,
<name of slide 2>,
<name of slide 3>,
<name of slide 4>,
...
<name of slide N>
]
}



In [78]:
answer = chatGPT.answerPropt(first, 500, newConv=True)

Tokens: 106
ChatGPT: Using: gpt-3.5-turbo
ChatGPT: Sending prompt


In [79]:
topics  = json.loads(answer)
topics = topics[list(topics.keys())[0]]
print(topics)

['Introduction', 'Founding of the Roman Republic', 'Early Roman Society', 'The Roman Constitution', 'The Roman Senate', 'The Roman Assemblies', 'The Roman Magistrates', 'The Roman Army', 'Roman Expansion', 'The Punic Wars', 'The Gracchi Brothers', 'The Social War', 'Sulla and the First Civil War', 'The Rise of Julius Caesar', 'The Second Triumvirate', 'The Assassination of Julius Caesar', 'The Rise of Augustus', "Augustus' Reforms", 'The Pax Romana', 'Fall of the Roman Republic']


In [80]:
outline = []
for top in topics:
    outline.append("    - " + top)
outline = "\n".join(outline)
print(outline)

    - Introduction
    - Founding of the Roman Republic
    - Early Roman Society
    - The Roman Constitution
    - The Roman Senate
    - The Roman Assemblies
    - The Roman Magistrates
    - The Roman Army
    - Roman Expansion
    - The Punic Wars
    - The Gracchi Brothers
    - The Social War
    - Sulla and the First Civil War
    - The Rise of Julius Caesar
    - The Second Triumvirate
    - The Assassination of Julius Caesar
    - The Rise of Augustus
    - Augustus' Reforms
    - The Pax Romana
    - Fall of the Roman Republic


In [81]:
with open('prompts.json') as f:
    prompts = json.load(f)

prompt = prompts["first_slide"].format(outline=outline, topOfSlide=topics[0], **options)

slides = []
slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
print(slides[0]["text"])

Tokens: 351
BingChat: Using: gpt-4
BingChat: Sending prompt
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
## Introduction
- The Roman Republic was the era of classical Roman civilization from **509 BC** to **27 BC**[^1^] [^2^]
- It began with the overthrow of the Roman Kingdom by a popular uprising and ended with the establishment of the Roman Empire by Augustus[^1^] [^2^]
- It was characterized by a mixed constitution of elected magistrates, a senate, and various assemblies[^1^] [^2^]
- It expanded its territory from the city of Rome to control most of the Mediterranean world through conquest and diplomacy[^1^] [^2^]
- It faced many internal and external challenges, such as social conflicts, civil wars, and foreign enemies[^1^] [^2^]
- It was a cultural mix of Latin, Etruscan, Greek, and other influences, especially in its religion and art[^1^] [^2^]

[^1^]: [Roman Republic - Wikipedia](https://en.wikipedia.org/wiki/Roman_Republic)
[^2^]: [Roman Republic | 

In [82]:
for i in range(1, len(topics)):
    gen = [slides[0]["text"]]
    for j in range(max(1, i-3), i-1):
        gen.append(slides[j]["text"])
    gen = "\n\n".join(gen)
    prompt = prompts["other_slide"].format(outline=outline, topOfSlide=topics[i], createdSlides=gen, **options)
    slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
    print(slides[i]["text"])


Tokens: 633
BingChat: Using: gpt-4
BingChat: Sending prompt
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
## Founding of the Roman Republic
- According to Roman tradition, the Republic began in **509 BCE** when a group of noblemen overthrew the last king of Rome[^1^] [^2^]
- The last king, **Lucius Tarquinius Superbus**, was a cruel tyrant who was opposed by his rival, **Lucius Junius Brutus**[^2^] [^5^]
- The Romans replaced the king with two **consuls** —rulers who had many of the same powers as the king but were elected to serve one-year terms[^1^] [^2^]
- The consuls were overseen by a **senate** and various **assemblies** that represented different classes of Roman society[^2^] [^4^]
- The Republic was established as a result of military defeat and foreign intervention by the **Etruscans**, who had dominated Rome and central Italy for centuries[^2^] [^3^]
- The Republic claimed to restore the ancient rights and customs of the Roman people that had been 

In [83]:
import deepl

with open(".DeepLtoken") as f:
    auth_key = f.read()
translator = deepl.Translator(auth_key)

In [84]:
import re

def formatSlide(text):
    lines = text.splitlines()
    ret = []
    for line in lines:
        if (line != "") and (line[0] != "["):
            line = re.sub("\*\*", "", line)
            line = re.sub(" ?\[.*?\] ?", "", line)
            ret.append(line)
    ret = "\n".join(ret)
    return ret

def transate(text):
    return translator.translate_text(text, target_lang="CS").text

In [85]:
pres_list = list(map(lambda x: transate(formatSlide(x["text"])), slides))
pres = "\n\n".join(pres_list)

# name = "pres {}.md"
# num = 1
# while os.path.exists(name.format(num)):
#     num+=1

# with open(name.format(num), "w", encoding="UTF-8") as f:
#     f.write(pres)

In [86]:
from pptx.enum.lang import MSO_LANGUAGE_ID
from pptx.util import Pt

def addSlide(power, text, language, font_size=20):
    lines = text.splitlines()

    lang_id = getattr(MSO_LANGUAGE_ID, language)

    lay = power.slide_layouts[1]
    slide = power.slides.add_slide(lay)

    slide.shapes.title.text = re.sub("^## +", "", lines[0])
    slide.shapes.title.text_frame.paragraphs[0].font.language_id = lang_id

    text_frame = slide.shapes.placeholders[1].text_frame

    first = True
    for line in lines[1:]:
        if first:
            p = text_frame.paragraphs[0]
            first = False
        else:
            p = text_frame.add_paragraph()
        p.text = re.sub("^ *- +", "", line)
        p.font.language_id = lang_id
        p.font.size = font_size
        if line[0] == "-":
            p.level = 0
        else:
            p.level = 1

    

power = Presentation("template.pptx")
power.core_properties.language = "CZECH"

nameOfPres = transate(options["top"])

lay = power.slide_layouts[0]
first = power.slides.add_slide(lay)
first.shapes.title.text = nameOfPres
first.placeholders[1].text= "Jan Slíva"

for slide in pres_list:
    addSlide(power, slide, "CZECH")
power.save(nameOfPres + ".pptx")

In [90]:
with open(nameOfPres + " sources.txt", "w", encoding="UTF-8") as f:
    for slide in slides:
        f.write(transate(slide["text"].splitlines()[0]) + "\n")
        for source in slide["sources"]:
            f.write("- " + source + "\n")
        f.write("\n")